In [2]:
import os
import numpy as np
import pandas as pd

from quantifier_recommender import QuantifierRecommender

quantifier_recommender = QuantifierRecommender()

# quantifier_recommender.construct_meta_table(datasets_path="./datasets/",
#                                             train_data_path="./data/train_data/",
#                                             test_data_path="./data/test_data/",
#                                             supervised=True)

# quantifier_recommender.evaluation_table.to_csv("./recommender_evaluation_table.csv", index=False)
# quantifier_recommender.meta_features_table.to_csv("./recommender_meta_features_table.csv", index=False)

df = pd.read_csv("./recommender_evaluation_table2.csv")

In [6]:
df.iloc[0]['quantifier']

str